In [1]:
import logging
import pandas as pd
from iso3166 import countries
from csv import QUOTE_NONNUMERIC
from unicef_schools_attribute_cleaning.pandas.dataframe_cleaner import dataframe_cleaner

# make INFO logs visible
logging.basicConfig(level=logging.INFO)

In [2]:
src_df = pd.read_csv('../../data/UNICE_schools_raw_2020_Jun/KZ-UNICEF-0-Government-0.csv', low_memory=False)
src_df

,school_id,admin1,admin3,name,educ_level,type_school,speed_connectivity,lat,lon
0,908,Акмолинская область,Aршалынский район,"Государственное учреждение""Центральная средняя...","малокомплектная школа, общеобразовательная школа",Коммунальная собственность,4.0,50.807155,72.113394
1,1468,Акмолинская область,Aршалынский район,Вячеславская СШ,"общеобразовательная школа, малокомплектная школа",Коммунальная собственность,8.0,50.996971,72.080400
2,1447,Акмолинская область,Aршалынский район,Берсуатская СШ,"общеобразовательная школа, малокомплектная школа",Коммунальная собственность,4.0,50.664100,72.117675
3,685,Акмолинская область,Aршалынский район,Нововладимировская СШ,"общеобразовательная школа, малокомплектная школа",Коммунальная собственность,4.0,51.286983,72.094823
4,854,Акмолинская область,Aршалынский район,СШ им.Абая,"общеобразовательная школа, малокомплектная школа",Коммунальная собственность,4.0,51.285193,72.090508
...,...,...,...,...,...,...,...,...,...
7405,12817,г.Шымкент,Каратауский район,СШ №72,общеобразовательная школа,Коммунальная собственность,10.0,42.276673,69.595917
7406,13399,г.Шымкент,Каратауский район,Школа- лицей №89,"общеобразовательная школа, школа-лицей",Коммунальная собственность,10.0,42.407250,69.608764
7407,12744,г.Шымкент,Каратауский район,СШ №69,общеобразовательная школа,Коммунальная собственность,10.0,42.402063,69.554381
7408,13307,г.Шымкент,Каратауский район,"Коммунальное государственное учреждение ""Общео...",общеобразовательная школа,Коммунальная собственность,10.0,42.371249,69.724163


In [3]:
# Problem: the type_school and educ_level columns have many Russian strings in them. In our schema, these are
# enumerated types: have specific allowed values. Therefore these need to be translated and
# then added to the enumerated type modules, e.g. SchoolType.py and EducationLevel.py. I added a few to the SchoolType module
# as an example (note: needs more translations)

In [4]:
src_df['type_school'].unique()

array(['Коммунальная собственность',
       'Собственность предприятий с участием государства (без иностранного участия)',
       'Республиканская собственность', 'Собственность граждан',
       'Собственность предприятий без государственного и иностранного участия',
       'Собственность иностранных юридических лиц',
       'Собственность международных организаций',
       'Собственность иностранных физических лиц',
       'Частная собственность',
       'Собственность совместных  предприятий с иностранным участием',
       'Собственность иностранных государств'], dtype=object)

In [5]:
len(src_df['educ_level'].unique())

110

In [6]:
# to avoid 7K warnings being printed by the dataframe_cleaner, wipe out the educ_level column until the translation problem is solved.
# example warning:
# WARNING:unicef_schools_attribute_cleaning.models.FuzzyMatchingEnum.EducationLevel:<aenum 'EducationLevel'> has no fuzzy match
# for 'общеобразовательная школа', choices = ['pre_primary', 'pre_básica', 'primary', 'básica', 'middle', 'media', 'secondary', 
# '_9_ybe', '_12_ybe', 'a_level', 'b_level', 'polytechnic', 'college', 'university', 'technical', 'vocational', 'vtc', 'tss', 
# 'disability', 'nursing']

src_df['educ_level'] = None

In [7]:
country = countries.get('KZ')
with open('kazakhstan_columns_report.txt', mode='w', encoding='utf-8') as filehandle:
    df = dataframe_cleaner(
        dataframe=src_df,
        country=country,
        removed_columns_report=filehandle,
        is_private=True,
        provider='UNICEF',
        provider_is_private=True
    )
df

INFO:unicef_schools_attribute_cleaning.pandas.dataframe_cleaner:copying dataframe...
INFO:unicef_schools_attribute_cleaning.pandas.dataframe_cleaner:standardizing column names...
INFO:unicef_schools_attribute_cleaning.pandas.standardize_column_names:uuid column not found, generating uuid4
INFO:unicef_schools_attribute_cleaning.pandas.standardize_column_names:renaming columns: {}
INFO:unicef_schools_attribute_cleaning.pandas.standardize_column_names:adding 37 columns from schema: ['admin0',
 'admin2',
 'admin4',
 'admin_code',
 'admin_id',
 'address',
 'address2',
 'phone_number',
 'person_contact',
 'email',
 'postal_code',
 'altitude',
 'gps_confidence',
 'date',
 'num_students',
 'num_teachers',
 'connectivity',
 'type_connectivity',
 'latency_connectivity',
 'availability_connectivity',
 'num_computers',
 'environment',
 'num_classrooms',
 'num_sections',
 'water',
 'electricity',
 'num_latrines',
 'description',
 'last_update',
 'tower_dist',
 'tower_type_service',
 'tower_type',
 

,country_code,admin0,admin1,admin2,admin3,admin4,admin_code,admin_id,name,address,...,tower_dist,tower_type_service,tower_type,tower_code,tower_latitude,tower_longitude,is_private,is_invalid,is_invalid_reason,uuid
0,KZ,Kazakhstan,Aqmola,Arshalynskiy,Aршалынский район,None,KAZ.2.2_1,"3.6,KAZ,GID_2=KAZ.2.2_1","Государственное учреждение""Центральная средняя...",None,...,None,None,None,None,None,None,True,False,None,acdb735d-7867-4283-835c-ead9eef869d1
1,KZ,Kazakhstan,Aqmola,Arshalynskiy,Aршалынский район,None,KAZ.2.2_1,"3.6,KAZ,GID_2=KAZ.2.2_1",Вячеславская СШ,None,...,None,None,None,None,None,None,True,False,None,9e0b82e6-c913-455d-a021-43c34dc14f2b
2,KZ,Kazakhstan,Aqmola,Arshalynskiy,Aршалынский район,None,KAZ.2.2_1,"3.6,KAZ,GID_2=KAZ.2.2_1",Берсуатская СШ,None,...,None,None,None,None,None,None,True,False,None,17a0f4d5-90f5-4b43-a5b5-aa7b17e95f3c
3,KZ,Kazakhstan,Aqmola,Arshalynskiy,Aршалынский район,None,KAZ.2.2_1,"3.6,KAZ,GID_2=KAZ.2.2_1",Нововладимировская СШ,None,...,None,None,None,None,None,None,True,False,None,edcfe8ce-6db2-488f-a20f-814d474fc0c4
4,KZ,Kazakhstan,Aqmola,Arshalynskiy,Aршалынский район,None,KAZ.2.2_1,"3.6,KAZ,GID_2=KAZ.2.2_1",СШ им.Абая,None,...,None,None,None,None,None,None,True,False,None,995e8f67-8ee1-4172-a9fe-9463be806e69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,KZ,Kazakhstan,South Kazakhstan,Shymkent,Каратауский район,None,KAZ.12.10_1,"3.6,KAZ,GID_2=KAZ.12.10_1",СШ №72,None,...,None,None,None,None,None,None,True,False,None,2433beba-144c-4da2-84f1-cc2ccc1a5ea1
7406,KZ,Kazakhstan,South Kazakhstan,Shymkent,Каратауский район,None,KAZ.12.10_1,"3.6,KAZ,GID_2=KAZ.12.10_1",Школа- лицей №89,None,...,None,None,None,None,None,None,True,False,None,6647c7d2-2f0f-4b3e-b31c-6f9d9bbe65a4
7407,KZ,Kazakhstan,South Kazakhstan,Shymkent,Каратауский район,None,KAZ.12.10_1,"3.6,KAZ,GID_2=KAZ.12.10_1",СШ №69,None,...,None,None,None,None,None,None,True,False,None,c692ef1e-586b-44a5-9208-57eaf309cd0c
7408,KZ,Kazakhstan,South Kazakhstan,Sayramskiy,Каратауский район,None,KAZ.12.9_1,"3.6,KAZ,GID_2=KAZ.12.9_1","Коммунальное государственное учреждение ""Общео...",None,...,None,None,None,None,None,None,True,False,None,58d66e42-b47e-4574-b932-d43438fe68d8


In [8]:
df.to_csv('kazakhstan_cleaned.csv', quoting=QUOTE_NONNUMERIC, index=False)

In [9]:
# open in LibreOffice, Excel, other
!open kazakhstan_cleaned.csv